Note cyril 13 avril 2021: 
I don't know why this notebook exist compare to main
Looks like it might be a first try to script a clean pipeline for rootlet analysis

Renamed as 'main_maybe'

import xlrd# Problem avec les Y soit du midline soit du edge

In [2]:
import xlrd

file = './data_excel_example/Analyse_de_l-orientation_des_rootlets_V2-2a _160427--Exploitation_clicks--.xlsx'


In [3]:
### DO YOU WANT: (True or False)

Compute_all_the_file_in_the_folder = False
# IF FALSE
# write the path of the image that you want to analyse:

IMAGE = './to_analyse/160330_WT_4.tif'



centriole_identification = False

orientation_prediction = False

orientation_compensation = True


# ANALYSIS



In [4]:
import import_ipynb

import torch
import torch.nn as nn

import xlrd  

from time import time, asctime

from glob import glob

import pickle

import cv2

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

from tools.Centriole_Characteristic import *
from tools.Extract_Experiment_Characteristic import *
from tools.CNN_Tools import *
from tools.Graphical_Tools import *
from tools.Centriole_Detection import *
from tools.Midline_Edge_Reformater import *
from tools.Extract_Experiment_Characteristic import *

#from tools.Worm_Segmentation import extract_worm_edge

importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Centriole_Characteristic.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/ToolBox.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Midline_Edge_Reformater.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Extract_Experiment_Characteristic.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/CNN_Tools.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Graphical_Tools.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Analysis_Tools.ipynb
importing Jupyter notebook from /home/cyril_b/projects/Planarians/tools/Centriole_Detection.ipynb


In [5]:
# ## Define files to treat
if Compute_all_the_file_in_the_folder:
    tif_list = glob('./to_analyse/*.tif')
    xls_list = glob('./to_analyse/*.xlsm')

else: 
    tif_list = [IMAGE]
    xls_list = [IMAGE[:-3] + 'xlsm']
       

for pathXls, pathImg in zip(xls_list, tif_list):
    print(f'{asctime()}')
    print(f'\t Analysis of image: {pathImg}')
    print(f'\t\t Centriole will be detected : {centriole_identification}')
    print(f'\t\t Centriole orientation will be predicted : {orientation_prediction}')
    print(f'\t\t Centriole orientation will be compensated : {orientation_compensation} \n')
    
    
    if centriole_identification:
        # Identification of the Centrioles
        img = cv2.imread(pathImg, cv2.IMREAD_UNCHANGED)
        
        # Detect Centriole with a combination of Differential of Gaussian and Otsu
        dog_img = dog_and_otsu(img)
        
        img_to_save = Image.fromarray(dog_img)
        newPath = pathImg[:-4] + '_dog_otsu.tif'
        img_to_save.save(newPath)
        print(f'{asctime()}: Dog & Otsu computed -> Saved')
        
        
        # Detect Centriole with a find maxima algorithm
        find_maxima_img = find_maxima(img)
        
        img_to_save = Image.fromarray(find_maxima_img)
        newPath = pathImg[:-4] + '_find_maxima.tif'
        img_to_save.save(newPath)
        print(f'{asctime()}: Find Maxima Computed -> Saved')
        
        
        # Combine the 2 detection method
        combine_img = dog_img*find_maxima_img
        
        img_to_save = Image.fromarray(combine_img)
        newPath = pathImg[:-4] + '_centriole_detected.tif'
        img_to_save.save(newPath)
        print(f'{asctime()}: Centriole detected -> Saved')


    
    if orientation_prediction:
        # Choose the device (cuda is faster but not always possible)
        if torch.cuda.is_available():                                  
            device = torch.device('cuda')
        else:
            device = torch.device('cpu')

        # Load the model that perform prediction
        model = VGG_Schmidtea(n_classes = 72).to(device)
        model.load_state_dict(torch.load('./weight/VGG_schmidtea_weight_classification.pth', map_location = device))
        model.eval()

        # Centriole orientation prediction, return a list of format ((X, Y), Angle)
        centrioleList = angle_prediction_from_img(pathImg, model, device)
        
        # Save the Data
        pathSave = pathImg[:-4] + '_predicted_angle.pickle'
        with open(pathSave, 'wb') as f:
            pickle.dump(centrioleList, f)
        
        print(f'{asctime()}: Centrioles angle predicted -> Save')


    if orientation_compensation:
        if orientation_prediction:
            pass
          
        else: 
            # Load the centriole List
            pathSave = pathImg[:-4] + '_predicted_angle.pickle'
            with open(pathSave, 'rb') as f:
                centrioleList = pickle.load(f) 
                
            print(f'{asctime()} \n\t Centriole location and orientation Loaded')
          
          
        # Shift the angle according to provided parameter
        # Note: the y axis is reversed (y = -y)
        centrioleList = centriole_shifter(centrioleList, pathExcel = pathXls)
        print('\t Centriole Shifted -> ', end = '')
    
        # Load Midline and edge in a list of list format: [x_mid, Y_mid, x_edg, Y_edg]
        # Note: the y axis is reversed (y = -y)
        worm = midline_edge_extractor(pathXls)
        print('Midline transformed -> ', end = '')
        midline = aggregate_segment_char(worm[0], worm[1], worm[2], worm[3], n_midline_seg = 50, n_sub_segment = 25, n_edge_seg = 200)
        print('Midline and Edge characterized')
        
        # Compensate centriole angle and compute there relative localization on the worm
        centrioleList = centriole_compensation(centrioleList, midline)
        
        pathSave = pathImg[:-4] + '_compensated_angle.pickle'
        with open(pathSave, 'wb') as f:
            pickle.dump(centrioleList, f)
        print(f'{asctime()}: Centrioles angle compensated and localization computed -> Save')



Tue Apr 13 21:02:17 2021
	 Analysis of image: ./to_analyse/160330_WT_4.tif
		 Centriole will be detected : False
		 Centriole orientation will be predicted : False
		 Centriole orientation will be compensated : True 



FileNotFoundError: [Errno 2] No such file or directory: './to_analyse/160330_WT_4_predicted_angle.pickle'

In [6]:
#######################################################
# Graphical representation of the results
# Code above is not mandatory
#######################################################

# If you want to see some specific centrioles, add them in the list
list_of_desired_centriole = [0]

# If you want to see the location of a specific coordinate (Write None if you don't want to see them )
X_coordinate = None
Y_coordinate = None

save_path = pathImg[:-4] + '_schema.tif'
Worm_And_Centriole(centrioleList, worm, list_of_desired_centriole, (X_coordinate, Y_coordinate), save = False, path = pathImg[:-4])

# Print The graph (worms segmented in 5 antero-posterior parts) + moving average + cstd
for i in range(5):
    print_a_antero_posterior_result(centrioleList, i, n_ante_post_segment = 5, a_lat_size = 0.1, a_lat_step = 0.05, save = False, path = pathImg[:-4])

# Overlap the analysed image with the identified and analyzed centriole represented as an arrow indicating the predicted angle
# The starting point of the arrow is the origin of the detected centriole
# The Ending point indicate the predicted orientation of the centriole

save_figure = True

# Each color correspond to a class of 5°
# So far the color or 'randomly' attributed for each class

# Define the length of the arrow
arrowLen = 5

# Compute as X/Y coordinates the property of the arrow
# WARNING: 18/11/2020. FOR AN UNKNOWN REASON, the angle is rotated by 90° -> I need to check why
DATA = []

for i in centrioleList:
    angle = i[1] + 90
    x = i[0][0]
    y = i[0][1]
    new_coord = [x-arrowLen*math.cos(math.radians(angle)), y-arrowLen*math.sin(math.radians(angle)), x+arrowLen*math.cos(math.radians(angle)), y+arrowLen*math.sin(math.radians(angle)), angle]
    DATA.append(new_coord)

DATA = np.array(DATA)

cmap = plt.cm.jet
cNorm  = colors.Normalize(vmin=np.min(DATA[:,4]), vmax=np.max(DATA[:,4]))
scalarMap = cmx.ScalarMappable(norm=cNorm,cmap=cmap)

plt.figure(figsize=(100,50))

plt.imshow(img)

for idx in range(0,len(DATA[:,1])):
    colorVal = scalarMap.to_rgba(DATA[idx,4])
    plt.arrow(DATA[idx,0],  #x1
              DATA[idx,1],  # y1
              DATA[idx,2]-DATA[idx,0], # x2 - x1
              DATA[idx,3]-DATA[idx,1], # y2 - y1
              color=colorVal)
if save_figure: 
    savePath = pathImg[:-4] + '_Detected_Angle.tif'
    plt.savefig(savePath)

"""
with open('./file_treated.csv', 'a', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(tif_list[i])"""
print(f"{asctime()}")
print("\n")


    


TypeError: bad operand type for unary -: 'list'

<Figure size 3600x1440 with 0 Axes>